In [1]:
import pandas as pd
import regex as re

from sklearn.model_selection import train_test_split

import ahocorasick

from tqdm import tqdm

In [2]:
SEP_TOKEN = ' ; '

In [3]:
df = pd.read_json ('train_t1_v1.jsonl', lines = True)
df.drop (columns = ['id', 'keywords'], inplace = True)
print (df.head ())

                                                text  \
0  ABBYY Retrieval & Morphology Engine\nВ сообщен...   
1  Речевые формулы в диалоге\nПредложенная класси...   
2  Географические названия и полнотекстовые докум...   
3  Методы автоматического построения специализиро...   
4  Закономерности построения дискурсивной последо...   

                                               label  
0  [[0, 35], [6, 15], [18, 35], [29, 35], [69, 88...  
1  [[0, 15], [18, 25], [74, 99], [134, 140], [175...  
2  [[0, 23], [26, 50], [54, 68], [169, 190], [181...  
3  [[7, 63], [34, 63], [54, 63], [92, 128], [119,...  
4  [[26, 57], [62, 76], [251, 265], [266, 280], [...  


In [4]:
def split_text (text, segments, delimiters = ['...', '.', '?!', '?', '!']):

    # Фильтрация
    #text = re.sub ('[“”]', '\"', text)

    # Создаем регулярное выражение для разделителей
    delimiters_pattern = '|'.join (map (re.escape, delimiters))
    
    # Разделяем текст на абзацы
    #paragraphs = text.split ('\n')
    paragraphs = re.split (f'(?<=\n[ ]*)', text)
    
    # Список для хранения предложений и их индексов
    sentences_with_segments = []
    
    current_start_index = 0
    # Обрабатываем каждый абзац
    for paragraph in paragraphs:
        # Разделяем абзац на предложения
        sentences = re.split (f'(?<=[{delimiters_pattern}] )(?=[A-ZА-ЯЁ])', paragraph)#.strip ())

        #print (f'\nРассматривается параграф: <{paragraph}>.')
        
        # Сопоставляем предложения с сегментами
        for sentence in sentences:
            #print ('> Предложение:', sentence)
            # Ищем индексы символов для текущего предложения
            start_index = current_start_index
            end_index = start_index + len (sentence)

            #print (f'Current sentence (from split): <{sentence}>')
            #print (f'Current sentence (by indices): <{text [start_index: end_index]}>')
            #print (f'Sentence start index: {start_index}, end index: {end_index}.')
            #print (f'Segments to match: {segments}')

            matched_segments = [
                text [start: end] for start, end in segments if start >= start_index and end <= end_index
            ]
            sentences_with_segments.append ((sentence, matched_segments))

            #current_start_index += len (sentence) + MAGIC_NUMBER
            current_start_index = end_index


            #print ('Термины:', matched_segments)
    
    return sentences_with_segments

In [5]:
parallel_text = []
parallel_label = []
#for i, row in tqdm (df.iterrows (), total = df.shape [0], desc = "Processing texts: "):
for i, row in df.iterrows ():
    text = row ['text']
    segments = row ['label']
    splitted = split_text (text, segments)
    for sentence, terms in splitted:
        parallel_text.append (sentence)
        constructed_label = ''
        for term in terms:
            constructed_label += term.strip () + SEP_TOKEN
        if len (constructed_label) > 0:
            parallel_label.append (constructed_label [: - len (SEP_TOKEN)])
        else:
            parallel_label.append ('')

In [6]:
all_train_terms = set ()

for line in parallel_label:
    
    all_train_terms |= set (line.split (SEP_TOKEN))

all_train_terms -= set ([''])
len (all_train_terms)

9761

In [7]:
def find_substrings (phrases, strings):
    # Создаем автомат
    A = ahocorasick.Automaton()
    
    # Добавляем все подстроки в автомат
    for idx, phrase in enumerate (phrases):
        A.add_word (phrase, (idx, phrase))
    
    # Завершаем построение автомата
    A.make_automaton ()
    
    results = {}
    
    # Ищем подстроки в каждой строке
    for s in strings:
        found = []
        for end_index, (idx, phrase) in A.iter (s):
            start_index = end_index - len (phrase) + 1

            # Не хочу цеплять подстроки внутри слов
            if start_index > 0 and s [start_index - 1].isalpha ():
                continue
            if end_index + 1 < len (s) and s [end_index + 1].isalpha ():
                continue

            found.append ((start_index, end_index + 1, phrase))
        
        results [s] = found
    
    return results

# Пример использования
#phrases = {"пример", "строка", "поиск"}
#strings = ["это пример строки для поиска", "другая строка без совпадений", "поиск в строке"]

#results = find_substrings (phrases, strings)

#for string, matches in results.items ():
#    print(f"В строке: '{string}' найдены совпадения: {matches}")

In [8]:
testres = find_substrings (all_train_terms, df ['text'].tolist ())
#testres = find_substrings (all_train_terms, parallel_text)

i = 0

count_pred_not_labels = 0
count_labels_not_pred = 0

for string, matches in testres.items ():
    slices = [[item [0], item [1]] for item in matches]
    slices.sort (key = lambda x: x [0])

    pred = set ([tuple (item) for item in slices])
    labl = set ([tuple (item) for item in df ['label'] [i]])

    diff = pred - labl
    for item in diff:
        #print (f'({i}) Item {item} - <{df ["text"] [i] [item [0]: item [1]]}> is in pred, but not in labels.')
        count_pred_not_labels += 1

    diff = labl - pred
    for item in diff:
        print (f'({i}) Item {item} - <{df ["text"] [i] [item [0]: item [1]]}> is in labels, but not in pred.')
        count_labels_not_pred += 1

    i += 1

print (f'Total {count_pred_not_labels} terms are in pred, but not labels.')
print (f'Total {count_labels_not_pred} terms are in labels, but not pred.')

(53) Item (408, 446) - <Грамматического словаря А.А. Зализняка> is in labels, but not in pred.
(96) Item (877, 905) - <“Анны Карениной” Л. Толстого> is in labels, but not in pred.
(107) Item (456, 511) - <«Грамматического словаря русского языка» А.А. Зализняка> is in labels, but not in pred.
(113) Item (316, 370) - <«Грамматический словарь русского языка» А.А. Зализняка> is in labels, but not in pred.
(144) Item (135, 167) - <пирамидальных сетей В.П. Гладуна> is in labels, but not in pred.
(151) Item (752, 789) - <стихотворения А.С. Пушкина “Памятник”> is in labels, but not in pred.
(175) Item (790, 852) - <Толковом словаре русского языка (Под ред. проф. Д. Н. Ушакова)> is in labels, but not in pred.
(187) Item (987, 1009) - <словаря А.А. Зализняка> is in labels, but not in pred.
(256) Item (348, 372) - <концепции М.А. Кронгауза> is in labels, but not in pred.
(332) Item (149, 166) - <говору с. Пустоша> is in labels, but not in pred.
(415) Item (405, 435) - <работе А. Б. Лорда «Сказите

In [9]:
# success log:
# Total X terms are in labels, but not pred/
# X: 993 -> 269 -> 613 (*_*) -> 1003 (-_-) -> 16 (˶ᵔ ᵕ ᵔ˶)

In [10]:
split_text (df ['text'] [5], df ['label'] [5])

[('К проблеме понимания несегментированного текста (на материале метеорологических телеграмм)\n',
  ['несегментированного текста', 'метеорологических телеграмм']),
 ('В фокусе внимания данной работы находится проблема восстановления структуры несегментированного текста. ',
  ['восстановления структуры несегментированного текста',
   'несегментированного текста']),
 ('Описывается процедура сегментации, которая осуществляет декомпозицию исходной лексической цепочки в последовательность тематически связных фрагментов, в рамках которых возможна семантическая интерпретация. ',
  ['сегментации',
   'декомпозицию',
   'лексической цепочки',
   'тематически связных фрагментов',
   'семантическая интерпретация']),
 ('Определяется нарративная структура текста в терминах типа сценария (прогрессивный или рекуррентный) и структуры эпизода (параллельная или последовательная). ',
  ['нарративная структура текста',
   'терминах',
   'сценария',
   'прогрессивный',
   'рекуррентный',
   'структуры эпиз

In [11]:
#paragraph = 'Предложенная классификация, как и многие другие лингвистические типологии, допускает пересечения. Например, идиома врать готово в последовательности реплик [- Честное слово] - Врать готово, с одной стороны, является комментарием, а с другой – обладает некоторыми характеристиками формул ответа: иллокутивно вынуждается предшествующей репликой и повторяет ее некоторые фонетические особенности. Кроме того, поскольку оценивается искренность предшествующей клятвы, данную идиому можно рассматривать и как формулу эпистемической модальности.'
text = "К проблеме понимания несегментированного текста (на материале метеорологических телеграмм)\nВ фокусе внимания данной работы находится проблема восстановления структуры несегментированного текста. Описывается процедура сегментации, которая осуществляет декомпозицию исходной лексической цепочки в последовательность тематически связных фрагментов, в рамках которых возможна семантическая интерпретация. Определяется нарративная структура текста в терминах типа сценария (прогрессивный или рекуррентный) и структуры эпизода (параллельная или последовательная). Тематическая сегментация и анализ нарративной структуры текста позволяют установить семантические связи (сферы действия) локативных и темпоральных модификаторов."

delimiters = ['...', '.', '?!', '?', '!']

delimiters_pattern = '|'.join (map (re.escape, delimiters))

paragraphs = re.split (f'(?<=\n)', text)

paragraphs

#sentences = re.split (f'(?<=[{delimiters_pattern}]) (?=[A-ZА-ЯЁ])', paragraph.strip ())

#sentences

['К проблеме понимания несегментированного текста (на материале метеорологических телеграмм)\n',
 'В фокусе внимания данной работы находится проблема восстановления структуры несегментированного текста. Описывается процедура сегментации, которая осуществляет декомпозицию исходной лексической цепочки в последовательность тематически связных фрагментов, в рамках которых возможна семантическая интерпретация. Определяется нарративная структура текста в терминах типа сценария (прогрессивный или рекуррентный) и структуры эпизода (параллельная или последовательная). Тематическая сегментация и анализ нарративной структуры текста позволяют установить семантические связи (сферы действия) локативных и темпоральных модификаторов.']

In [12]:
def raw_splitter (text, delimiters = ['...', '.', '?!', '?', '!']):

    delimiters_pattern = '|'.join (map (re.escape, delimiters))
    paragraphs = re.split (f'(?<=\n[ ]*)', text)
    
    sentences_with_indices = []
    current_start_index = 0

    for paragraph in paragraphs:

        sentences = re.split (f'(?<=[{delimiters_pattern}] )(?=[A-ZА-ЯЁ])', paragraph)#.strip ())

        for sentence in sentences:

            start_index = current_start_index
            end_index = start_index + len (sentence)

            sentences_with_indices.append ((sentence, (start_index, end_index)))

            current_start_index = end_index
    
    return sentences_with_indices

def one_finder (text, phrases):

    A = ahocorasick.Automaton ()
    
    for idx, phrase in enumerate (phrases):
        A.add_word (phrase, (idx, phrase))
    
    A.make_automaton ()
    
    found = []
    for end_index, (idx, phrase) in A.iter (text):
        start_index = end_index - len (phrase) + 1

        if start_index > 0 and text [start_index - 1].isalpha ():
            continue
        if end_index + 1 < len (text) and text [end_index + 1].isalpha ():
            continue

        found.append ((start_index, end_index + 1, phrase))
    
    return found

def answerer (text, term_set = all_train_terms):
    sentences_w_ind = raw_splitter (text)

    answers = []
    for sentence, (start, end) in sentences_w_ind:
        found = one_finder (sentence, term_set)
        answers += [[item [0] + start, item [1] + start] for item in found]

    return answers

def comparator (pred, labl):

    pred = set ([tuple (item) for item in pred])
    labl = set ([tuple (item) for item in labl])

    true_positives = len (pred & labl)
    false_positives = len (pred - labl)
    false_negatives = len (labl - pred)

    return true_positives, false_positives, false_negatives

def metricator (preds, labels):

    tps_sum = 0
    fps_sum = 0
    fns_sum = 0 

    for i in range (len (labels)):

        true_positives, false_positives, false_negatives = comparator (preds [i], labels [i])

        tps_sum += true_positives   # Истинно положительные
        fps_sum += false_positives     # Ложноположительные
        fns_sum += false_negatives     # Ложноотрицательные

    precision = tps_sum / (tps_sum + fps_sum) if (tps_sum + fps_sum) > 0 else 0
    recall = tps_sum / (tps_sum + fns_sum) if (tps_sum + fns_sum) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

In [13]:
answerer (df ['text'] [0])

[[6, 15],
 [0, 35],
 [18, 35],
 [29, 35],
 [69, 88],
 [112, 119],
 [147, 159],
 [194, 226],
 [205, 226],
 [258, 280],
 [274, 280],
 [283, 307],
 [300, 307],
 [314, 323],
 [308, 343],
 [326, 343],
 [337, 343],
 [372, 391]]

In [14]:
metricator ([answerer (df ['text'] [0])], [df ['label'] [0]])

(0.9444444444444444, 1.0, 0.9714285714285714)

In [15]:
'''
answers_bydict = []

for i, row in tqdm (df.iterrows ()):
    answers_bydict.append (answerer (row ['text']))

labels = df ['label'].tolist ()

metricator (answers_bydict, labels)
'''

"\nanswers_bydict = []\n\nfor i, row in tqdm (df.iterrows ()):\n    answers_bydict.append (answerer (row ['text']))\n\nlabels = df ['label'].tolist ()\n\nmetricator (answers_bydict, labels)\n"

In [16]:
train_data_txt, temp_data_txt, train_labels_txt, temp_labels_txt = train_test_split (df ['text'], df ['label'], test_size = 0.2, random_state = 14)
val_data_txt, test_data_txt, val_labels_txt, test_labels_txt = train_test_split (temp_data_txt, temp_labels_txt, test_size = 0.5, random_state = 14)

In [17]:
only_train_terms = set ()

train_data = train_data_txt.tolist ()
train_labels = train_labels_txt.tolist ()

for i in range (len (train_data)):

    terms = [train_data [i] [item [0]: item [1]] for item in train_labels [i]]
    
    only_train_terms |= set (terms)

only_train_terms -= set ([''])
len (only_train_terms)

8048

In [18]:
val_data = val_data_txt.tolist ()
val_labels = val_labels_txt.tolist ()

only_val_terms = set ()
for i in range (len (val_data)):
    terms = [val_data [i] [item [0]: item [1]] for item in val_labels [i]]
    only_val_terms |= set (terms)
only_val_terms -= set ([''])
len (only_val_terms)

answers_bydict = []
for item in tqdm (val_data):
    answers_bydict.append (answerer (item, only_train_terms))

print (metricator (answers_bydict, val_labels))
print (len (only_train_terms & only_val_terms))

100%|██████████| 85/85 [00:04<00:00, 20.99it/s]

(0.6165980795610425, 0.4928728070175439, 0.5478366849482023)
489


In [19]:
val_data = test_data_txt.tolist ()
val_labels = test_labels_txt.tolist ()

only_test_terms = set ()
for i in range (len (val_data)):
    terms = [val_data [i] [item [0]: item [1]] for item in val_labels [i]]
    only_test_terms |= set (terms)
only_test_terms -= set ([''])
len (only_test_terms)

answers_bydict = []
for item in tqdm (val_data):
    answers_bydict.append (answerer (item, only_train_terms))

print (metricator (answers_bydict, val_labels))
print (len (only_train_terms & only_test_terms))

100%|██████████| 85/85 [00:04<00:00, 19.78it/s]

(0.5824175824175825, 0.4836285560923242, 0.5284457478005864)
464


In [20]:
print (f'Терминов в обучающем наборе: {len (only_train_terms)}.')
print (f'Терминов в валидационном наборе: {len (only_val_terms)}.')
print (f'Терминов в тестовом наборе: {len (only_test_terms)}.')

print ()

print (f'При этом пересечений обучающего и валидационного: {len (only_train_terms & only_val_terms)},')
print (f'         пересечений обучающего и тестового:      {len (only_train_terms & only_test_terms)}')
print (f'     пересечений валидационного и тестового:      {len (only_val_terms & only_test_terms)}')

Терминов в обучающем наборе: 8048.
Терминов в валидационном наборе: 1352.
Терминов в тестовом наборе: 1350.

При этом пересечений обучающего и валидационного: 489,
         пересечений обучающего и тестового:      464
     пересечений валидационного и тестового:      181


In [22]:

def label_constructor (labels):
    res = []
    for label in labels:
        one_label = []
        for start, end, cls in label:
            one_label.append ([start, end])
        res.append (one_label)
    return res

df_2 = pd.read_json ('./test_data/test1_t12_full_v2.jsonl', lines = True, encoding = 'utf8')
df_2 = df_2 [['text', 'label']]
print (df_2.head ())

df_2 ['label'] = label_constructor (df_2 ['label'])

                                                text  \
0  АВТОМАТИЧЕСКИЙ АНАЛИЗ ТОНАЛЬНОСТИ ТЕКСТОВ НА О...   
1  InBASE: ТЕХНОЛОГИЯ ПОСТРОЕНИЯ ЕЯ-ИНТЕРФЕЙСОВ К...   
2  Выражение уважительности с помощью личных мест...   
3  ДА ЧЕРТ ЛИ В ДЕТАЛЯХ?.. МЕРА ДЛЯ ОЦЕНКИ СОВПАД...   
4  КОРПУСНАЯ ОЦЕНКА СОЧЕТАЕМОСТИ СЛОВ С ИСПОЛЬЗОВ...   

                                               label  
0  [[0, 33, specific], [22, 33, specific], [52, 7...  
1  [[0, 6, nomen], [19, 44, specific], [30, 44, s...  
2  [[0, 24, specific], [35, 53, specific], [42, 5...  
3  [[51, 70, specific], [61, 70, specific], [116,...  
4  [[0, 16, specific], [17, 34, specific], [30, 3...  


In [23]:
only_train_terms = set ()

train_data = df ['text'].tolist ()
train_labels = df ['label'].tolist ()

for i in range (len (train_data)):

    terms = [train_data [i] [item [0]: item [1]] for item in train_labels [i]]
    
    only_train_terms |= set (terms)

only_train_terms -= set ([''])
len (only_train_terms)

9777

In [24]:
val_data = df_2 ['text'].tolist ()
val_labels = df_2 ['label'].tolist ()

val_data_txt, test_data_txt, val_labels_txt, test_labels_txt = train_test_split (val_data, val_labels, test_size = 0.5, random_state = 14)

only_val_terms = set ()
for i in range (len (val_data_txt)):
    terms = [val_data_txt [i] [item [0]: item [1]] for item in val_labels_txt [i]]
    only_val_terms |= set (terms)
only_val_terms -= set ([''])
len (only_val_terms)

answers_bydict = []
for item in tqdm (val_data_txt):
    answers_bydict.append (answerer (item, only_train_terms))

print (metricator (answers_bydict, val_labels_txt))
print (len (only_train_terms & only_val_terms))

100%|██████████| 56/56 [00:07<00:00,  7.03it/s]

(0.5832476875642343, 0.4797125950972105, 0.5264378478664193)
448


In [25]:
only_val_terms = set ()
for i in range (len (test_data_txt)):
    terms = [test_data_txt [i] [item [0]: item [1]] for item in test_labels_txt [i]]
    only_val_terms |= set (terms)
only_val_terms -= set ([''])
len (only_val_terms)

answers_bydict = []
for item in tqdm (test_data_txt):
    answers_bydict.append (answerer (item, only_train_terms))

print (metricator (answers_bydict, test_labels_txt))
print (len (only_train_terms & only_val_terms))

100%|██████████| 57/57 [00:08<00:00,  6.43it/s]

(0.5896964121435143, 0.4874524714828897, 0.5337218984179851)
499


In [26]:
def get_set (tensor, ground_truth = True, tokenizer = tokenizer):

    separator: str
    if SEP_TOKEN == '▁<extra_id_0>': separator = '<extra_id_0>'
    else: separator = SEP_TOKEN

    res: set

    if ground_truth:
        eos_idx = (tensor == 1).nonzero ()
        if eos_idx.numel () > 0:
            eos_idx = int (eos_idx [0] [0])
        else:
            eos_idx = len (tensor)
        seq = tensor [:eos_idx]
    
    else:
        seq = tensor [tensor != 0]
        seq = seq [seq != 1]
    
    txt = tokenizer.decode (seq)
    res = set ([item.strip () for item in txt.split (separator)])

    if len (res) > 1:
        res -= set ([''])

    return res


def sanity_check (preds, labels, to_print = False):

    tps_sum = 0
    fps_sum = 0
    fns_sum = 0 

    for i in range (len (labels)):
        predicted_set = get_set (preds [i], ground_truth = False)
        true_set = get_set (labels [i])

        if to_print: print (f'True: {true_set}\nPred: {predicted_set}')

        tps_sum += len (true_set & predicted_set)  # Истинно положительные
        fps_sum += len (predicted_set - true_set)     # Ложноположительные
        fns_sum += len (true_set - predicted_set)     # Ложноотрицательные

    precision = tps_sum / (tps_sum + fps_sum) if (tps_sum + fps_sum) > 0 else 0
    recall = tps_sum / (tps_sum + fns_sum) if (tps_sum + fns_sum) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1_score

NameError: name 'tokenizer' is not defined

In [ ]:
import torch

SEQ_MAX_LENGTH = 150

def predict_with_model (texts, model, tokenizer):

    model.eval ()

    predictions = []

    for text in tqdm (texts):

        sentences_w_ind = raw_splitter (text)

        answers = []
        for sentence, (start, end) in sentences_w_ind:

            sentence_tokenized = tokenizer (sentence, padding = 'max_length', truncation = True, max_length = SEQ_MAX_LENGTH, return_tensors = 'pt')

            with torch.no_grad ():

                input_ids = sentence_tokenized ['input_ids'].to (model.device)
                attention_mask = sentence_tokenized ['attention_mask'].to (model.device)

                out = model.generate (input_ids = input_ids, attention_mask = attention_mask)

                term_set = get_set (out)

            found = one_finder (sentence, term_set)
            answers += [[item [0] + start, item [1] + start] for item in found]

        predictions.append (answers)

    return (predictions)